# 0) Imports

In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
# %matplotlib inline

In [94]:
import os
import numpy as np
import pandas as pd
from time import time

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# 1) Personal tools and solvers

In [95]:
from utils import is_symmetric, is_pos_def, is_commutative, res_norm_pr, rel_err_pr
from plots import plot_conv, plot_eigenvals, plot_conv_plotly
from results import ResultPR

from gen_data import gen_artifical_pr
from adi_peaceman_rachford import adi_pr, sap_adi_pr
from shift_parameters import shifts_pr, shifts_w

from save_load import save, save_artificial_pr, load_artificial_pr

### Parameters to select

In [96]:
N = 8000  # dimension
spectra = "close"

overwrite = True  # overwrite matrices
n_iter_moving_shifts = 15  # number iterations PR and W shifts
n_iter_cst_shifts = 40  # number iterations cst shifts
n_iter = 50  # number iterations BSPR

is_plot_spectrum = False  # plot spectrum of matrices A and B


# Plots title
title = f"Peaceman-Rachford problem N={N}"

# 2) Generate input matrices and vector

## Check that generated matrices are symmetric, positive-definite and commute

In [97]:
# np.random.seed(0)

# N_small = 10
# H_small, V_small, _, _ = gen_artifical_pr(N_small)

In [98]:
# print("H and V are symmetric:", is_symmetric(H_small) and is_symmetric(V_small))
# print("H and V are positive-definite:", is_pos_def(H_small) and is_pos_def(V_small))
# print("H and V commute:", is_commutative(H_small, V_small))

In [99]:
# print(np.linalg.eigvalsh(H_small))
# print(np.linalg.eigvalsh(V_small))

## Load or generate data

In [100]:
filename = f"artificial_pr_{N}_{spectra}.npz"
folder = os.path.join(os.getcwd(), "datasets/pr/")
path = os.path.join(folder, filename)

if os.path.exists(path) and not overwrite:
    H, V, b, u_true = load_artificial_pr(N, spectra)
    print("Matrices loaded from\n", path)
else:
    np.random.seed(0)
    H, V, b, u_true = gen_artifical_pr(N, spectra)
    save_artificial_pr(H, V, b, u_true, spectra)
    print("Matrices saved in\n", path)
    
m = H.shape[0]
print("Shape of H :", m)

Matrices saved in
 /home/claire/nidham/thomas_jefferson_fund/sketch_proj_ADI/code/sketched_adi/datasets/pr/artificial_pr_8000_close.npz
Shape of H : 8000


In [101]:
# Check that u_true is a solution
if u_true.size == 0:
    print("Solution not provided")
else:
    if res_norm_pr(u_true, H, V, b) < 1e-3:
        print("The problem is well posed")
#         u_direct = np.linalg.solve(H + V, b) # computed solution
#         print(res_norm_pr(u_direct, H, V, b))
#         print(np.linalg.norm(u_direct - u_true)) # u_true: ground truth solution
    else:
        print("The problem does not have a solution")
        u_true = np.array([])

The problem is well posed


In [102]:
# Estimate eigenvalue interval
t0 = time()
eig_val_H = np.linalg.eigvalsh(H)
eig_val_V = np.linalg.eigvalsh(V)

all_eig_val = np.concatenate((eig_val_H, eig_val_V))
alpha = np.min(all_eig_val)
beta = np.max(all_eig_val)
t_eigvalsh = time() - t0

print(
    f"Spectrum of H: [{np.min(eig_val_H):.5f}, {np.max(eig_val_H):.5f}]"
    f" => condition number (H) = {np.linalg.cond(H):.0f}"
)
print(
    f"Spectrum of V: [{np.min(eig_val_V):.5f}, {np.max(eig_val_V):.5f}]"
    f" => condition number (V) = {np.linalg.cond(V):.0f}"
)

print(f"Eigenvalue interval [alpha, beta] = [{alpha}, {beta}]\n")
print(f"Time numpy eigvalsh: {t_eigvalsh:.2} s\n")

print(f"Condition number (H + V) = {np.linalg.cond(H + V):.0f}")

Spectrum of H: [0.00010, 1.00010] => condition number (H) = 10001
Spectrum of V: [0.00015, 1.00015] => condition number (V) = 6668
Eigenvalue interval [alpha, beta] = [9.999999999963536e-05, 1.0001499999999885]

Time numpy eigvalsh: 1.1e+02 s

Condition number (H + V) = 8001


In [103]:
# close
init = 1e-4
mu = .01
power = 1e0
print(init + np.exp(-mu*1000) / power, init + np.exp(-mu*0) / power)
print(init + np.exp(-mu*2500) / power, init + np.exp(-mu*0) / power)
print(init + np.exp(-mu*5000) / power, init + np.exp(-mu*0) / power, "\n")

init = 1.5e-4
print(init + np.exp(-mu*1000) / power, init + np.exp(-mu*0) / power)
print(init + np.exp(-mu*2500) / power, init + np.exp(-mu*0) / power)
print(init + np.exp(-mu*5000) / power, init + np.exp(-mu*0) / power)

0.00014539992976248485 1.0001
0.00010000001388794387 1.0001
0.0001 1.0001 

0.00019539992976248485 1.00015
0.00015000001388794384 1.00015
0.00015 1.00015


## Remark
H + V = H - (- V), sp(H) and sp(-V) must be separated enough
a = min eig (H), b = max eig (H)
c = min eig (-V), d = max eig (-V)

gam = (c - a )*( d - b )/( c - b )/( d - a ); % Cross - ratio of a ,b ,c , d
=> Measures the "difficulty" of the problem 

AX - XB = F when the eigenvalues of A ( B ) are in [a , b ] and
% the eigenvalues of B ( A ) are in [c , d ]

In [104]:
if is_plot_spectrum:
    # Plot eigenvalues decrease
    plot_eigenvals(eig_val_H, eig_val_V, "pr")

# 3) ADI to solve Peaceman-Rachford problem

## Solvers parameters

In [105]:
results = []

# a) Test ADI with Peaceman-Rachford shifts

In [106]:
# Compute PR shifts
t0 = time()
p_pr, q_pr = shifts_pr(alpha, beta, n_iter_moving_shifts)
t_init = time() - t0
    
u_adi_pr, t_adi_pr, iter_adi_pr, epoch_adi_pr = adi_pr(
    H, V, b, 
    n_iter=n_iter_moving_shifts, 
    p=p_pr, q=q_pr,
    store_every=int(n_iter_moving_shifts/10),
    verbose=True,
)

# Taking initialization into account
# t_eigvalsh : time to compute spectrum bounds
t_adi_pr += t_eigvalsh + t_init

result_adi_pr = ResultPR(
    "PR-pr_shifts", 
    u_adi_pr, t_adi_pr, iter_adi_pr, epoch_adi_pr,
    u_true,
)
result_adi_pr.compute_errors(H, V, b)

results.append(result_adi_pr)

print(f"\nRelative residual of last iterate: {result_adi_pr.rel_res[-1]:.2e}")
if not result_adi_pr.rel_err.size == 0:
    print(f"Relative error of last iterate: {result_adi_pr.rel_err[-1]:.2e}")

--------------------------------------------
  Iteration   |    Epoch     |   Time (s)   
--------------------------------------------
      1       |      1       |   1.06e+01   
      2       |      2       |   2.16e+01   
      3       |      3       |   3.29e+01   
      4       |      4       |   4.42e+01   
      5       |      5       |   5.57e+01   
      6       |      6       |   6.96e+01   
      7       |      7       |   8.34e+01   
      8       |      8       |   9.71e+01   
      9       |      9       |   1.18e+02   
      10      |      10      |   1.35e+02   
      11      |      11      |   1.46e+02   
      12      |      12      |   1.57e+02   
      13      |      13      |   1.70e+02   
      14      |      14      |   1.84e+02   
      15      |      15      |   1.98e+02   

Relative residual of last iterate: 1.53e-04
Relative error of last iterate: 7.09e-05


# b) Test ADI with Wachspress shifts

In [107]:
# With Wachspress shifts
t0 = time()
p_w, q_w = shifts_w(alpha, beta, n_iter_moving_shifts)
t_init = time() - t0

u_adi_w, t_adi_w, iter_adi_w, epoch_adi_w = adi_pr(
    H, V, b, 
    n_iter=n_iter_moving_shifts, 
    p=p_w, q=q_w,
    store_every=int(n_iter_moving_shifts/10),
    verbose=True,
)

# Taking initialization into account
# t_eigvalsh : time to compute spectrum bounds
t_adi_w += t_eigvalsh + t_init

result_adi_w = ResultPR(
    "PR-w_shifts", 
    u_adi_w, t_adi_w, iter_adi_w, epoch_adi_w,
    u_true,
)
result_adi_w.compute_errors(H, V, b)

results.append(result_adi_w)

print(f"\nRelative residual of last iterate: {result_adi_w.rel_res[-1]:.2e}")
if not result_adi_w.rel_err.size == 0:
    print(f"Relative error of last iterate: {result_adi_w.rel_err[-1]:.2e}")

--------------------------------------------
  Iteration   |    Epoch     |   Time (s)   
--------------------------------------------
      1       |      1       |   2.28e+01   
      2       |      2       |   3.84e+01   
      3       |      3       |   4.96e+01   
      4       |      4       |   6.09e+01   
      5       |      5       |   7.37e+01   
      6       |      6       |   8.76e+01   
      7       |      7       |   1.01e+02   
      8       |      8       |   1.24e+02   
      9       |      9       |   1.41e+02   
      10      |      10      |   1.53e+02   
      11      |      11      |   1.64e+02   
      12      |      12      |   1.77e+02   
      13      |      13      |   1.91e+02   
      14      |      14      |   2.05e+02   
      15      |      15      |   2.27e+02   

Relative residual of last iterate: 1.89e-05
Relative error of last iterate: 2.81e-06


# c) Test ADI with constant shifts p = q = beta (upper bound of eigenvalues)

In [108]:
def normalize(x):
    fac = abs(x).max()
    x_n = x / x.max()
    return fac, x_n

In [109]:
t0 = time()
x = np.ones(m)
for i in range(10):
    x = np.dot(H, x)
    lambda_max_H, x = normalize(x)

x = np.ones(m)
for i in range(10):
    x = np.dot(V, x)
    lambda_max_V, x = normalize(x)

t_power_method = time() - t0


beta_estimate = max(lambda_max_H, lambda_max_V)
print(f"Beta estimate: {beta_estimate}")
print(f"Beta: {beta}")

print(f"Time power method: {t_power_method:.2} s")

Beta estimate: 0.9592586193680881
Beta: 1.0001499999999885
Time power method: 0.21 s


In [110]:
# With constant shifts
u_adi_cst, t_adi_cst, iter_adi_cst, epoch_adi_cst = adi_pr(
    H, V, b, 
    n_iter=n_iter_cst_shifts, 
#     p=beta, q=beta,
    p=beta_estimate, q=beta_estimate,
    store_every=int(n_iter_cst_shifts/10),
    verbose=True
)

# Taking initialization into account
# t_power_method : time to compute spectrum upper bounds
t_adi_cst += t_power_method

result_adi_cst = ResultPR(
    "PR-cst", 
    u_adi_cst, t_adi_cst, iter_adi_cst, epoch_adi_cst,
    u_true,
)
result_adi_cst.compute_errors(H, V, b)

results.append(result_adi_cst)

print(f"\nRelative residual of last iterate: {result_adi_cst.rel_res[-1]:.2e}")
if not result_adi_cst.rel_err.size == 0:
    print(f"Relative error of last iterate: {result_adi_cst.rel_err[-1]:.2e}")

--------------------------------------------
  Iteration   |    Epoch     |   Time (s)   
--------------------------------------------
      4       |      4       |   5.15e+01   
      8       |      8       |   1.20e+02   
      12      |      12      |   1.69e+02   
      16      |      16      |   2.36e+02   
      20      |      20      |   2.88e+02   
      24      |      24      |   3.54e+02   
      28      |      28      |   4.11e+02   
      32      |      32      |   4.71e+02   
      36      |      36      |   5.37e+02   
      40      |      40      |   5.90e+02   

Relative residual of last iterate: 4.26e-03
Relative error of last iterate: 9.38e-01


# d) Sketch and Project ADI with shift parameters

In [111]:
fracs = [.1, .25]
for size_frac in fracs:
    sketch_size = int(size_frac*N) # run same number of epochs
    n_iter_sto = int(n_iter/size_frac)
    print(f"Problem of size: {N} | sketch dimension: {sketch_size} ({int(100*size_frac)}% of rows)")
    print(f"Number of iterations: {n_iter_sto}")

    beta_array = beta*np.ones(n_iter_sto)
    np.random.seed(0)
    u_sketch, t_sketch, iter_sketch, epoch_sketch = sap_adi_pr(
        H, V, b, 
        n_iter=n_iter_sto, 
        p=beta_array, q=beta_array,
        sketch_size=sketch_size, 
        store_every=int(n_iter_sto/10),
        verbose=True,
    )
    
    # Taking initialization into account
    # t_power_method : time to compute spectrum upper bounds
    t_sketch += t_power_method

    result_sketched_adi_cst_shifts = ResultPR(
        "BSPR-cst-" + str(size_frac), 
        u_sketch, t_sketch, iter_sketch, epoch_sketch,
        u_true
    )
    result_sketched_adi_cst_shifts.compute_errors(H, V, b)

    results.append(result_sketched_adi_cst_shifts)

    print(f"\nRelative residual of last iterate: {result_sketched_adi_cst_shifts.rel_res[-1]:.2e}")
    if not result_sketched_adi_cst_shifts.rel_err.size == 0:
        print(f"Relative error of last iterate: {result_sketched_adi_cst_shifts.rel_err[-1]:.2e}")
    print("\n")

Problem of size: 8000 | sketch dimension: 800 (10% of rows)
Number of iterations: 500
--------------------------------------------
  Iteration   |    Epoch     |   Time (s)   
--------------------------------------------
      50      |     5.0      |   2.80e+01   
     100      |     10.0     |   6.74e+01   
     150      |     15.0     |   9.18e+01   
     200      |     20.0     |   1.20e+02   
     250      |     25.0     |   1.59e+02   
     300      |     30.0     |   1.87e+02   
     350      |     35.0     |   2.12e+02   
     400      |     40.0     |   2.45e+02   
     450      |     45.0     |   2.85e+02   
     500      |     50.0     |   3.09e+02   

Relative residual of last iterate: 3.78e-03
Relative error of last iterate: 9.39e-01


Problem of size: 8000 | sketch dimension: 2000 (25% of rows)
Number of iterations: 200
--------------------------------------------
  Iteration   |    Epoch     |   Time (s)   
--------------------------------------------
      20      |    

# 4) Compare convergence plots: iterations and time

In [112]:
# plot_conv_plotly(results, title, prob="pr", x_axis="iter")
plot_conv_plotly(results, title, prob="pr", x_axis="epoch")
plot_conv_plotly(results, title, prob="pr", x_axis="time")

# 5) Save the results in csv and corresponding plots

In [114]:
# to run twice to avoid MathJax message on the plot
exp_name = f"artificial_N_{N}_spectra_{spectra}_a_tilde_{alpha:.2e}_b_tilde_{beta:.2e}"
save(results, title, "pr", exp_name)

Full path: /home/claire/nidham/thomas_jefferson_fund/sketch_proj_ADI/code/sketched_adi/outputs/pr/artificial_N_8000_spectra_close_a_tilde_1.00e-04_b_tilde_1.00e+00
Save results in csv
Save plots
